# FleetTrack Tech Residency 40

In [1]:
import numpy as np
import pandas as pd
import time
import json
import random

### 4 hours of data for one vehicle

In [ ]:
car1 = pd.DataFrame({"vehicle_id": ['c1'] * 24,
                    "timestamp": pd.date_range(start='2023-10-01 00:09:00', periods=24, freq='10min'),
                    "speed_mph": np.round(np.random.uniform(0, 70, 24), 2),})
print(car1)

In [ ]:
# Random Walk (simple realistic variation)
np.random.seed(42)

n = 24
timestamps = pd.date_range(start='2023-10-01 00:09:00', periods=n, freq='10min')

# Start around 30 mph and add small random changes
speeds = [30]
for _ in range(1, n):
    change = np.random.uniform(-5, 5)  # small change per step
    new_speed = max(0, min(70, speeds[-1] + change))  # keep in range [0,70]
    speeds.append(new_speed)

car1 = pd.DataFrame({
    "vehicle_id": ["c1"] * n,
    "timestamp": timestamps,
    "speed_mph": np.round(speeds, 2),
    "check_engine": [0] * (n - 2) + [1, 0] #simulate a check engine event
})
print(car1)


In [ ]:
### Location Database
# Base coordinates roughly around Phoenix, AZ
base_lat = 33.45
base_lon = -112.07

# Slight random offsets for nearby locations (~a few miles)
def random_nearby_coordinates(base_lat, base_lon, delta=0.05):
    return (
        base_lat + random.uniform(-delta, delta),
        base_lon + random.uniform(-delta, delta)
    )

# Example fake addresses
streets = [
    "123 Main St",
    "456 Oak Ave",
    "789 Pine Rd",
    "101 Maple Blvd",
    "202 Elm St"
]

cities = ["Phoenix", "Tempe", "Scottsdale", "Mesa", "Glendale"]

# Generate 5 unique location dicts
locations = []
for i in range(5):
    lat, lon = random_nearby_coordinates(base_lat, base_lon)
    entry = {
        "location_id": f"Location_{i+1}",
        "address": {
            "street": streets[i],
            "city": cities[i],
            "state": "AZ",
            "zip": str(85000 + i)  # simple sequential ZIPs
        },
        "latitude": lat,
        "longitude": lon
    }
    locations.append(entry)

# Save to JSON file
with open("location_data.json", "w") as f:
    json.dump(locations, f, indent=2)


[{'location_id': 'Location_1',
  'address': {'street': '123 Main St',
   'city': 'Phoenix',
   'state': 'AZ',
   'zip': '85000'},
  'latitude': 33.4126642094734,
  'longitude': -112.07944811954908},
 {'location_id': 'Location_2',
  'address': {'street': '456 Oak Ave',
   'city': 'Tempe',
   'state': 'AZ',
   'zip': '85001'},
  'latitude': 33.48875319222395,
  'longitude': -112.03755148020659},
 {'location_id': 'Location_3',
  'address': {'street': '789 Pine Rd',
   'city': 'Scottsdale',
   'state': 'AZ',
   'zip': '85002'},
  'latitude': 33.40214625788509,
  'longitude': -112.08301005507717},
 {'location_id': 'Location_4',
  'address': {'street': '101 Maple Blvd',
   'city': 'Mesa',
   'state': 'AZ',
   'zip': '85003'},
  'latitude': 33.41568584255849,
  'longitude': -112.10992747945114},
 {'location_id': 'Location_5',
  'address': {'street': '202 Elm St',
   'city': 'Glendale',
   'state': 'AZ',
   'zip': '85004'},
  'latitude': 33.41876329329107,
  'longitude': -112.09583864248516}]